# Dictionary-based `aux_operators`

<br/>
<br/>
<br/>
<br/>

Max Rossmannek

PhD Student, Quantum Applications Group, IBM Research Zurich

Recently, Qiskit Terra added support for dictionary-based `aux_operators` to their (Minimum)Eigensolver algorithms (https://github.com/Qiskit/qiskit-terra/pull/6870; credits: @CisterMoke). This means, we can now do both of the following:

In [1]:
from qiskit.opflow import I, X, Y, Z

h2_op = (
    -1.052373245772859 * (I ^ I)
    + 0.39793742484318045 * (I ^ Z)
    - 0.39793742484318045 * (Z ^ I)
    - 0.01128010425623538 * (Z ^ Z)
    + 0.18093119978423156 * (X ^ X)
)

In [2]:
list_aux_ops = [
    2.0 * (I ^ I),
    0.5 * (
        (I ^ I) + (Z ^ Z) + (Y ^ Y) - (X ^ X)
    ),
]

dict_aux_ops = {
    "AuxOp1": 2.0 * (I ^ I),
    "AuxOp2": 0.5 * (
        (I ^ I) + (Z ^ Z) + (Y ^ Y) - (X ^ X)
    ),
}

In [3]:
from qiskit.algorithms import NumPyMinimumEigensolver

solver = NumPyMinimumEigensolver()

In [4]:
list_result = solver.compute_minimum_eigenvalue(h2_op, aux_operators=list_aux_ops)
print(list_result)

{   'aux_operator_eigenvalues': [((1.9999999999999996+0j), 0.0), (0.0, 0.0)],
    'eigenstate': VectorStateFn(Statevector([ 1.38777878e-16+1.29020059e-17j,
              7.22856695e-01+6.81936898e-01j,
             -8.11307233e-02-7.65380388e-02j,
              5.55111512e-16-1.38777878e-16j],
            dims=(2, 2)), coeff=1.0, is_measurement=False),
    'eigenvalue': -1.857275030202379}


In [5]:
dict_result = solver.compute_minimum_eigenvalue(h2_op, aux_operators=dict_aux_ops)
print(dict_result)

{   'aux_operator_eigenvalues': {   'AuxOp1': ((1.9999999999999987+0j), 0.0),
                                    'AuxOp2': (0.0, 0.0)},
    'eigenstate': VectorStateFn(Statevector([ 0.        -2.77555756e-17j, -0.63914825-7.60952858e-01j,
              0.0717356 +8.54064936e-02j,  0.        +5.55111512e-17j],
            dims=(2, 2)), coeff=1.0, is_measurement=False),
    'eigenvalue': -1.8572750302023824}


This is very interesting for us application developers because we can significantly improve our code quality.
Take the following as an example of how we had to handle default `aux_operators` in our stack:

```python
        for aux_op_eigenvalues in aux_operator_eigenvalues:
            if aux_op_eigenvalues is None:
                continue

            if len(aux_op_eigenvalues) >= 6:
                dipole_moment = []
                for moment in aux_op_eigenvalues[3:6]:  # <- hard-coded indices !!!
                    if moment is not None:
                        dipole_moment += [moment[0].real]
                    else:
                        dipole_moment += [None]
```

And this is just one example. Such hard-coded indices appeared in multiple places across our stack.

With the dictionary-support and in combination with Qiskit Nature's relatively new `Property` framework (https://qiskit.org/documentation/nature/apidocs/qiskit_nature.properties.html#module-qiskit_nature.properties), we can significantly improve this implementation:

```python
        for aux_op_eigenvalues in aux_operator_eigenvalues:
            if aux_op_eigenvalues is None:
                continue

            axes_order = {"x": 0, "y": 1, "z": 2}
            dipole_moment = [None] * 3
            
            for prop in iter(self):
                moment = aux_op_eigenvalues.get(prop.name, None)
                if moment is not None:
                    dipole_moment[axes_order[prop._axis]] = moment[0].real
```

We can now observe a difference when using the `ElectronicStructureProblem`:

In [6]:
from qiskit_nature.drivers import Molecule
from qiskit_nature.drivers.second_quantization import (
    ElectronicStructureMoleculeDriver,
    ElectronicStructureDriverType,
)

molecule = Molecule(geometry=[("H", (0.0, 0.0, 0.0)), ("H", (0.0, 0.0, 0.735))])
driver = ElectronicStructureMoleculeDriver(molecule, "sto3g", driver_type=ElectronicStructureDriverType.PYSCF)

/home/oss/Files/Qiskit/.direnv/python-3.9.7/lib64/python3.9/site-packages/pyscf/lib/misc.py:46: H5pyDeprecationWarning: Using default_file_mode other than 'r' is deprecated. Pass the mode to h5py.File() instead.
  h5py.get_config().default_file_mode = 'a'


In [7]:
from qiskit_nature.problems.second_quantization import ElectronicStructureProblem

problem = ElectronicStructureProblem(driver)

You will now be able to tell a difference, upon inspecting the `SecondQuantizedOp`s generated by the problem instance:

In [8]:
problem.second_q_ops()

[FermionicOp([('+_0 -_1 +_2 -_3', (0.18093119978423136+0j)), ('+_0 -_1 -_2 +_3', (-0.18093119978423156+0j)), ('-_0 +_1 +_2 -_3', (-0.18093119978423156+0j)), ('-_0 +_1 -_2 +_3', (0.18093119978423164+0j)), ('+_...)], register_length=4, display_format='sparse'),
 FermionicOp([('+_0 -_0', (1+0j)), ('+_1 -_1', (1+0j)), ('+_2 -_2', (1+0j)), ('+_3 -_3', (1+0j))], register_length=4, display_format='sparse'),
 FermionicOp([('+_0 -_1 -_2 +_3', (1+0j)), ('-_0 +_1 +_2 -_3', (1+0j)), ('+_3 -_3', (0.75+0j)), ('+_2 -_2', (0.75+0j)), ('+_2 -_2 +_3 -_3', (0.5+0j)), ('+_1 -_1', (0.75+0j)), ('+_1 -_1 +_3 -_3', (-1.5+0j)), (...)], register_length=4, display_format='sparse'),
 FermionicOp([('+_0 -_0', (0.5+0j)), ('+_1 -_1', (0.5+0j)), ('+_2 -_2', (-0.5+0j)), ('+_3 -_3', (-0.5+0j))], register_length=4, display_format='sparse'),
 FermionicOp([('', 0j)], register_length=4, display_format='sparse'),
 FermionicOp([('', 0j)], register_length=4, display_format='sparse'),
 FermionicOp([('+_0 -_1', (0.9278334704592

In [9]:
from qiskit_nature import settings
settings.dict_aux_operators = True

In [10]:
problem.second_q_ops()

{'ParticleNumber': FermionicOp([('+_0 -_0', (1+0j)), ('+_1 -_1', (1+0j)), ('+_2 -_2', (1+0j)), ('+_3 -_3', (1+0j))], register_length=4, display_format='sparse'),
 'ElectronicEnergy': FermionicOp([('+_0 -_1 +_2 -_3', (0.18093119978423136+0j)), ('+_0 -_1 -_2 +_3', (-0.18093119978423156+0j)), ('-_0 +_1 +_2 -_3', (-0.18093119978423156+0j)), ('-_0 +_1 -_2 +_3', (0.18093119978423164+0j)), ('+_...)], register_length=4, display_format='sparse'),
 'DipoleMomentX': FermionicOp([('', 0j)], register_length=4, display_format='sparse'),
 'DipoleMomentY': FermionicOp([('', 0j)], register_length=4, display_format='sparse'),
 'DipoleMomentZ': FermionicOp([('+_0 -_1', (0.9278334704592323+0j)), ('-_0 +_1', (-0.9278334704592324+0j)), ('+_2 -_3', (0.9278334704592323+0j)), ('-_2 +_3', (-0.9278334704592324+0j)), ('+_3 -_3', (0.6944743507776605+0j)), ('...)], register_length=4, display_format='sparse'),
 'AngularMomentum': FermionicOp([('+_0 -_1 -_2 +_3', (1+0j)), ('-_0 +_1 +_2 -_3', (1+0j)), ('+_3 -_3', (0.7

In the remainder of the stack, you will not be able to tell them apart:

In [11]:
from qiskit_nature.converters.second_quantization import QubitConverter
from qiskit_nature.mappers.second_quantization import JordanWignerMapper

converter = QubitConverter(JordanWignerMapper())

In [12]:
from qiskit_nature.algorithms.ground_state_solvers import GroundStateEigensolver
from qiskit_nature.algorithms.ground_state_solvers.minimum_eigensolver_factories import NumPyMinimumEigensolverFactory

solver = GroundStateEigensolver(converter, NumPyMinimumEigensolverFactory(use_default_filter_criterion=True))

In [13]:
result = solver.solve(problem)
print(result)

=== GROUND STATE ENERGY ===
 
* Electronic ground state energy (Hartree): -1.857275030202
  - computed part:      -1.857275030202
~ Nuclear repulsion energy (Hartree): 0.719968994449
> Total ground state energy (Hartree): -1.137306035753
 
=== MEASURED OBSERVABLES ===
 
  0:  # Particles: 2.000 S: 0.000 S^2: 0.000 M: 0.000
 
=== DIPOLE MOMENTS ===
 
~ Nuclear dipole moment (a.u.): [0.0  0.0  1.3889487]
 
  0: 
  * Electronic dipole moment (a.u.): [0.0  0.0  1.3889487]
    - computed part:      [0.0  0.0  1.3889487]
  > Dipole moment (a.u.): [0.0  0.0  0.0]  Total: 0.0
                 (debye): [0.0  0.0  0.0]  Total: 0.0
 


However, in the future this will allow us to allow users to directly supply custom properties, which are to be evaluated alongside the default ones, within the problem defintion:
```python
my_custom_property = CustomProperty(...)

problem = ElectronicStructureProblem(driver, aux_properties=[my_custom_property])
```
> If `aux_properties` is a list, the `Property.name` attribute will be used as the dictionary-key within the stack.

In [14]:
import qiskit.tools.jupyter
%qiskit_version_table
%qiskit_copyright